<a href="https://colab.research.google.com/github/mantzaris/teddyCode/blob/main/debug_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2022-12-15 21:56:53 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [ ]:
import Pkg; Pkg.add("Plots");
import Pkg; Pkg.add("StatsBase");
import Pkg; Pkg.add("InvertedIndices")
import Pkg; Pkg.add("Flux")
import Pkg; Pkg.add("OneHotArrays")
import Pkg; Pkg.add("Graphs")
import Pkg; Pkg.add("GraphPlot")
import Pkg; Pkg.add("Distributions")
Pkg.add("PyCall")

In [2]:
using Random
using InvertedIndices
using SparseArrays
using StatsBase
using LinearAlgebra
using Plots
using BenchmarkTools
using Flux
using PyCall
using OneHotArrays
using Statistics
using Graphs
using GraphPlot
using Distributions
np = pyimport("numpy")

PyObject <module 'numpy' from '/usr/local/lib/python3.8/dist-packages/numpy/__init__.py'>

In [3]:
#convert adj to S marix
function A2S(AdjMat)
    AdjMat += I #add the identity to the diagonal, add self-loops
    diag = Diagonal(vec(sum(AdjMat,dims=2) .^ (-1/2)))
    return (diag) * AdjMat * (diag) #return the normalized S matrix
end


function create_adjacency_matrix(num, distro)
  Random.seed!(10)
  ER_tmp = erdos_renyi( num , 10*(num) )
  BA_tmp = barabasi_albert( num , 8 )
  SF_tmp = static_scale_free( num , 8*(num) , 4 )
  WS_tmp = erdos_renyi( num , 10*(num) ) #barabasi_albert( NN_tmp , 5 )
  blocks_tmp = blockdiag( blockdiag( blockdiag(ER_tmp,BA_tmp),SF_tmp ), WS_tmp )
    
  #now add some edges between the blocks that are the communities
  for bb in 1:Int(round(num/10))
    for b1 in 0:3
      for b2 in 0:3
        if(b1 < b2)
          range1 = randperm(num)[1] + b1*num
          range2 = randperm(num)[1] + b2*num
          add_edge!( blocks_tmp , range1 , range2 )
        end
      end
    end
  end
  density_nn = Graphs.density(blocks_tmp)
  adj = Matrix(adjacency_matrix(blocks_tmp))

 
  d1 = distro([11 ,5 ,1])
  d2 = distro([2, 10, 8])
  d3 = distro([2 ,5, 4]) 
  c1 = Categorical( [0.5,0.25,0.25] )
  c2 = Categorical( [0.15,0.15,0.7] )
  c3 = Categorical( [0.5,0.5,0] )

  xd1 = rand( d1 , num )
   xc1 = onehotbatch( rand( c1 , num ) , 1:3 )
   x1a = vcat( xd1 , xc1 )'
   xd1 = rand( d1 , num )
   xc1 = onehotbatch( rand( c1 , num ) , 1:3 )
   x1b = vcat( xd1 , xc1 )'
   xd2 = rand( d2 , num )
   xc2 = onehotbatch( rand( c2 , num ) , 1:3 )
   x2 = vcat( xd2 , xc2 )'
   xd3 = rand( d3 , num )
   xc3 = onehotbatch( rand( c3 , num ) , 1:3 )
   x3 = vcat( xd3 , xc3 )'
   xc3 = onehotbatch( rand( c3 , num ) , 1:3 )
   x3 = vcat( xd3 , xc3 )'

  X = vcat( x1a , x1b , x2 , x3 )
  y1a = onehotbatch( 1*ones(num) , 1:2 )'
  y1b = onehotbatch( 1*ones(num) , 1:2 )'
  y2 = onehotbatch( 2*ones(num) , 1:2 )'
  y3 = onehotbatch( 2*ones(num) , 1:2 )'
  Y = vcat(y1a, y1b, y2, y3)
  Y_to_use = vcat(1*ones(num), 1*ones(num), 2*ones(num), 2*ones(num))
  return adj, X, Y_to_use
end 

function split_matrix(data, at)
    Random.seed!(1)
    n =size(data)[2]
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    trained = data[:,train_idx]
    tested = data[:,test_idx]
    return(trained, tested)
end


function load_and_train_2(SX_, yhot_, train_x_, train_y_, test_x_, test_y_)
  model_ = Chain( Dense( size(SX_, 1) => size(yhot_, 1)) , softmax)
  loss(x, y) = Flux.crossentropy(model_(x), y)
  opt = Adam(0.01)
  pars = Flux.params(model_)
  data = Flux.DataLoader((SX_, yhot_) , batchsize = 10 , shuffle = true)
  epochs_ = Int64[]
  loss_on_train_ = Float32[]
  loss_on_test_ = Float32[]
  for epoch in 1:500
    Flux.train!(loss, pars, data ,opt)
    push!(epochs_, epoch)
    push!(loss_on_train_, loss(train_x_, train_y_))
    push!(loss_on_test_, loss(test_x_, test_y_))
  end 
  return(epochs_, loss_on_train_, loss_on_test_, model_)
end 


function train_3(num, method, distro)
  for k in 0:5
    ad, x, y = create_adjacency_matrix(num, distro)
    yhot = onehotbatch(y, [1, 2])
    S = A2S(ad)
    SX = S^k * x
    SX
    SX = SX'
    train_x, test_x = split_matrix(SX, 0.7)
    train_y, test_y = split_matrix(yhot, 0.7)
    if method == 0
      #println("Training with raw data")
      epochs, loss_on_train, loss_on_test,  model = load_and_train_2(SX, yhot, train_x, train_y, test_x, test_y)
    end

      accuracy = round(mean( onecold( model(train_x), [1, 2] ) .== onecold(train_y, [1, 2]) ) * 100, digits = 3)
      println("Accuracy: ", accuracy, "%", ", k = $k, diatribution = $distro")
    
    
      #plot(epochs, loss_on_train, label = "loss on train")
      #plot!(epochs, loss_on_test, label = "loss on test")
  end
end 

train_3 (generic function with 1 method)

In [4]:
l = [MvNormalCanon, MvNormal, MvLogNormal, Dirichlet ]
for i in l
  train_3(250, 0, i)
end 

Accuracy: 59.143%, k = 0, diatribution = MvNormalCanon
Accuracy: 79.714%, k = 1, diatribution = MvNormalCanon
Accuracy: 98.429%, k = 2, diatribution = MvNormalCanon
Accuracy: 100.0%, k = 3, diatribution = MvNormalCanon
Accuracy: 100.0%, k = 4, diatribution = MvNormalCanon
Accuracy: 100.0%, k = 5, diatribution = MvNormalCanon
Accuracy: 55.0%, k = 0, diatribution = MvNormal
Accuracy: 77.857%, k = 1, diatribution = MvNormal
Accuracy: 97.429%, k = 2, diatribution = MvNormal
Accuracy: 100.0%, k = 3, diatribution = MvNormal
Accuracy: 100.0%, k = 4, diatribution = MvNormal
Accuracy: 100.0%, k = 5, diatribution = MvNormal
Accuracy: 56.857%, k = 0, diatribution = MvLogNormal
Accuracy: 96.286%, k = 1, diatribution = MvLogNormal
Accuracy: 49.429%, k = 2, diatribution = MvLogNormal
Accuracy: 49.429%, k = 3, diatribution = MvLogNormal
Accuracy: 49.429%, k = 4, diatribution = MvLogNormal
Accuracy: 49.429%, k = 5, diatribution = MvLogNormal
Accuracy: 98.857%, k = 0, diatribution = Dirichlet
Accuracy: